In [78]:
import numpy as np
import sympy as sym

In [89]:
l_aa,l_ab,l_ac = sym.symbols('l_aa,l_ab,l_ac', real = True)
l_ba,l_bb,l_bc = sym.symbols('l_ba,l_bb,l_bc', real = True)
l_ca,l_cb,l_cc = sym.symbols('l_ca,l_cb,l_cc', real = True)

v_a,v_b,v_c = sym.symbols('v_a,v_b,v_c', real = True)
di_a,di_b,di_c = sym.symbols('di_a,di_b,di_c', real = True)
i_a,i_b,i_c = sym.symbols('i_a,i_b,i_c', real = True)
R_s,omega_r,R_c,tau = sym.symbols('R_s,omega_r,R_c,tau', real = True)
phi_ar,phi_br,phi_cr = sym.symbols('phi_ar,phi_br,phi_cr', real = True)


In [80]:
eq_i_a =  l_aa*di_a + l_ab*di_b + l_ac*di_c - v_a + R_s * i_a + phi_ar * omega_r    
eq_i_b =  l_ba*di_a + l_bb*di_b + l_bc*di_c - v_b + R_s * i_b + phi_br * omega_r
eq_i_c =  l_ca*di_a + l_cb*di_b + l_cc*di_c - v_c + R_s * i_c + phi_cr * omega_r

In [81]:
sol = sym.solve([eq_i_a,eq_i_b,eq_i_c],[di_a,di_b,di_c])

In [82]:
sym.simplify(sol[di_a])

(-R_s*i_a*l_bb*l_cc + R_s*i_a*l_bc*l_cb + R_s*i_b*l_ab*l_cc - R_s*i_b*l_ac*l_cb - R_s*i_c*l_ab*l_bc + R_s*i_c*l_ac*l_bb - l_ab*l_bc*omega_r*phi_cr + l_ab*l_bc*v_c + l_ab*l_cc*omega_r*phi_br - l_ab*l_cc*v_b + l_ac*l_bb*omega_r*phi_cr - l_ac*l_bb*v_c - l_ac*l_cb*omega_r*phi_br + l_ac*l_cb*v_b - l_bb*l_cc*omega_r*phi_ar + l_bb*l_cc*v_a + l_bc*l_cb*omega_r*phi_ar - l_bc*l_cb*v_a)/(l_aa*l_bb*l_cc - l_aa*l_bc*l_cb - l_ab*l_ba*l_cc + l_ab*l_bc*l_ca + l_ac*l_ba*l_cb - l_ac*l_bb*l_ca)

In [83]:
str(sol[di_a])

'(-R_s*i_a*l_bb*l_cc + R_s*i_a*l_bc*l_cb + R_s*i_b*l_ab*l_cc - R_s*i_b*l_ac*l_cb - R_s*i_c*l_ab*l_bc + R_s*i_c*l_ac*l_bb - l_ab*l_bc*omega_r*phi_cr + l_ab*l_bc*v_c + l_ab*l_cc*omega_r*phi_br - l_ab*l_cc*v_b + l_ac*l_bb*omega_r*phi_cr - l_ac*l_bb*v_c - l_ac*l_cb*omega_r*phi_br + l_ac*l_cb*v_b - l_bb*l_cc*omega_r*phi_ar + l_bb*l_cc*v_a + l_bc*l_cb*omega_r*phi_ar - l_bc*l_cb*v_a)/(l_aa*l_bb*l_cc - l_aa*l_bc*l_cb - l_ab*l_ba*l_cc + l_ab*l_bc*l_ca + l_ac*l_ba*l_cb - l_ac*l_bb*l_ca)'

In [37]:
sym.expand(eq_i_a*i_a + eq_i_b*i_b + eq_i_c*i_c)

R_s*i_a**2 + R_s*i_b**2 + R_s*i_c**2 + di_a*i_a*l_aa + di_a*i_b*l_ba + di_a*i_c*l_ca + di_b*i_a*l_ab + di_b*i_b*l_bb + di_b*i_c*l_cb + di_c*i_a*l_ac + di_c*i_b*l_bc + di_c*i_c*l_cc + i_a*omega_r*phi_ar - i_a*v_a + i_b*omega_r*phi_br - i_b*v_b + i_c*omega_r*phi_cr - i_c*v_c

In [46]:
import scipy.io as sio
import matplotlib.pyplot as plt
import pydae.plot_tools as pltt
import sys

colors = pltt.set_style(plt)

In [55]:
%matplotlib widget

In [56]:
data = sio.loadmat('./simulink/pmsm.mat')

In [57]:
data['torque'].shape

(21, 21, 31)

In [121]:
def abc2dq(a,b,c,theta):
    
    d =  np.cos(theta)*a + np.cos(theta-2/3*np.pi)*b + np.cos(theta-4/3*np.pi)*c
    q = -np.sin(theta)*a - np.sin(theta-2/3*np.pi)*b - np.sin(theta-4/3*np.pi)*c
        
    return d,q

def dq2abc(d,q,theta):
    
    a = np.cos(theta)*d - np.sin(theta)*q
    b = np.cos(theta-2/3*np.pi)*d - np.sin(theta-2/3*np.pi)*q
    c = np.cos(theta-4/3*np.pi)*d - np.sin(theta-4/3*np.pi)*q
        
    return a,b,c

# i_a,i_b,i_c,theta_r -> inv_park -> id,i_q,theta_r -> matrix -> phi_d,phi_q -> park -> phi_a,phi_b,phi_c
# parc phi_a/ parc i_a = (phi_a(i+1) - phi_a(i-1))/(i_a(i+1) - i_a(i-1))

i_a = 200.0
i_b =   0.0
i_c =   0.0
theta = 0.0

i_d,i_q = abc2dq(i_a,i_b,i_c,theta)

#dq2abc(data['fluxD'],data['fluxQ'],data['angleVec'][0])


#for i_d in data['idVec'][0]:
#    for i_q in data['iqVec'][0]:
#        i_a,i_b,i_c = dq2abc(i_d,i_q,theta)
#        print(i_d,i_q)
idx_i_d = np.searchsorted(data['idVec'][0],i_d)
idx_i_q = np.searchsorted(data['iqVec'][0],i_q)
idx_theta = np.searchsorted(data['angleVec'][0],theta)

In [123]:
from scipy.interpolate import interpn

In [122]:
data['fluxD'][idx_i_d,idx_i_q,idx_theta]

0.30409187775

In [144]:
points = (data['idVec'][0],data['iqVec'][0],data['angleVec'][0])

Di = 10
Dtheta = 1.0
i_a = 200.0
i_b =   0.0
i_c =   0.0
theta = 0.0

i_a_1 = i_a - Di
i_d,i_q = abc2dq(i_a_1,i_b,i_c,theta)
point = (i_d,i_q,theta)
phi_d_1 = interpn(points,data['fluxD'],point)
phi_q_1 = interpn(points,data['fluxQ'],point)

phi_a_1,phi_b_1,phi_c_1 = dq2abc(phi_d_1,phi_q_1,theta)

i_a_2 = i_a + Di
i_d,i_q = abc2dq(i_a_2,i_b,i_c,theta)
point = (i_d,i_q,theta)
phi_d_2 = interpn(points,data['fluxD'],point)
phi_q_2 = interpn(points,data['fluxQ'],point)

phi_a_2,phi_b_2,phi_c_2 = dq2abc(phi_d_2,phi_q_2,theta)

l_aa = (phi_a_1 - phi_a_2)/(i_a_1 - i_a_2)
l_ab = (phi_b_1 - phi_b_2)/(i_a_1 - i_a_2)
l_ac = (phi_c_1 - phi_c_2)/(i_a_1 - i_a_2)


In [195]:
I_a = np.arange(-100,100+20,20)
I_b = np.copy(I_a)
I_c = np.copy(I_a)

N_I = I_a.shape[0]
N_th = data['angleVec'][0].shape[0]
l_aa = np.zeros((N_I,N_I,N_I,N_th))
l_ab = np.zeros((N_I,N_I,N_I,N_th))
l_ac = np.zeros((N_I,N_I,N_I,N_th))
l_ba = np.zeros((N_I,N_I,N_I,N_th))
l_bb = np.zeros((N_I,N_I,N_I,N_th))
l_bc = np.zeros((N_I,N_I,N_I,N_th))
l_ca = np.zeros((N_I,N_I,N_I,N_th))
l_cb = np.zeros((N_I,N_I,N_I,N_th))
l_cc = np.zeros((N_I,N_I,N_I,N_th))
phi_r_a_s = np.zeros((N_I,N_I,N_I,N_th))
phi_r_b_s = np.zeros((N_I,N_I,N_I,N_th))
phi_r_c_s = np.zeros((N_I,N_I,N_I,N_th))

In [210]:
Di = 10
Dtheta = np.deg2rad(0.1)

for th in range(N_th):
    print(th)
    for i in range(N_I):
        for j in range(N_I):  
            for k in range(N_I):  
                i_a,i_b,i_c,theta = I_a[i],I_b[j],I_c[k],np.deg2rad(data['angleVec'][0][th])
                
                # i_a
                i_a_1 = i_a - Di
                i_d,i_q = abc2dq(i_a_1,i_b,i_c,theta)
                point = (i_d,i_q,np.rad2deg(theta))
                phi_d_1 = interpn(points,data['fluxD'],point)
                phi_q_1 = interpn(points,data['fluxQ'],point)

                phi_a_1,phi_b_1,phi_c_1 = dq2abc(phi_d_1,phi_q_1,theta)

                i_a_2 = i_a + Di
                i_d,i_q = abc2dq(i_a_2,i_b,i_c,theta)
                point = (i_d,i_q,np.rad2deg(theta))
                phi_d_2 = interpn(points,data['fluxD'],point)
                phi_q_2 = interpn(points,data['fluxQ'],point)

                phi_a_2,phi_b_2,phi_c_2 = dq2abc(phi_d_2,phi_q_2,theta)

                l_aa[i,j,k,th] = (phi_a_1 - phi_a_2)/(i_a_1 - i_a_2)
                l_ba[i,j,k,th] = (phi_b_1 - phi_b_2)/(i_a_1 - i_a_2)
                l_ca[i,j,k,th] = (phi_c_1 - phi_c_2)/(i_a_1 - i_a_2)

                # i_b
                i_b_1 = i_b - Di
                i_d,i_q = abc2dq(i_a,i_b_1,i_c,theta)
                point = (i_d,i_q,np.rad2deg(theta))
                phi_d_1 = interpn(points,data['fluxD'],point)
                phi_q_1 = interpn(points,data['fluxQ'],point)

                phi_a_1,phi_b_1,phi_c_1 = dq2abc(phi_d_1,phi_q_1,theta)

                i_b_2 = i_b + Di
                i_d,i_q = abc2dq(i_a,i_b_2,i_c,theta)
                point = (i_d,i_q,np.rad2deg(theta))
                phi_d_2 = interpn(points,data['fluxD'],point)
                phi_q_2 = interpn(points,data['fluxQ'],point)

                phi_a_2,phi_b_2,phi_c_2 = dq2abc(phi_d_2,phi_q_2,theta)

                l_ab[i,j,k,th] = (phi_a_1 - phi_a_2)/(i_b_1 - i_b_2)
                l_bb[i,j,k,th] = (phi_b_1 - phi_b_2)/(i_b_1 - i_b_2)
                l_cb[i,j,k,th] = (phi_c_1 - phi_c_2)/(i_b_1 - i_b_2)  
                
                # i_c
                i_c_1 = i_c - Di
                i_d,i_q = abc2dq(i_a,i_b,i_c_1,theta)
                point = (i_d,i_q,np.rad2deg(theta))
                phi_d_1 = interpn(points,data['fluxD'],point)
                phi_q_1 = interpn(points,data['fluxQ'],point)

                phi_a_1,phi_b_1,phi_c_1 = dq2abc(phi_d_1,phi_q_1,theta)

                i_c_2 = i_c + Di
                i_d,i_q = abc2dq(i_a,i_b,i_c_2,theta)
                point = (i_d,i_q,np.rad2deg(theta))
                phi_d_2 = interpn(points,data['fluxD'],point)
                phi_q_2 = interpn(points,data['fluxQ'],point)

                phi_a_2,phi_b_2,phi_c_2 = dq2abc(phi_d_2,phi_q_2,theta)

                l_ac[i,j,k,th] = (phi_a_1 - phi_a_2)/(i_c_1 - i_c_2)
                l_bc[i,j,k,th] = (phi_b_1 - phi_b_2)/(i_c_1 - i_c_2)
                l_cc[i,j,k,th] = (phi_c_1 - phi_c_2)/(i_c_1 - i_c_2) 
                
                # theta_r
                if th == 0:
                    theta_1 = theta - 0*Dtheta
                    theta_2 = theta + Dtheta
                if th > 0 and th < (N_th-1):
                    theta_1 = theta - Dtheta
                    theta_2 = theta + Dtheta
                if th >= (N_th-1):
                    theta_1 = theta - Dtheta
                    theta_2 = theta
                    
                i_d,i_q = abc2dq(i_a,i_b,i_c,theta_1)
                point = (i_d,i_q,np.rad2deg(theta_1))
                phi_d_1 = interpn(points,data['fluxD'],point)
                phi_q_1 = interpn(points,data['fluxQ'],point)

                phi_a_1,phi_b_1,phi_c_1 = dq2abc(phi_d_1,phi_q_1,theta_1)

                i_d,i_q = abc2dq(i_a,i_b,i_c,theta_2)
                point = (i_d,i_q,np.rad2deg(theta_2))
                phi_d_2 = interpn(points,data['fluxD'],point)
                phi_q_2 = interpn(points,data['fluxQ'],point)

                phi_a_2,phi_b_2,phi_c_2 = dq2abc(phi_d_2,phi_q_2,theta_2)

                phi_r_a_s[i,j,k,th] = (phi_a_1 - phi_a_2)/(theta_1 - theta_2)
                phi_r_b_s[i,j,k,th] = (phi_b_1 - phi_b_2)/(theta_1 - theta_2)
                phi_r_c_s[i,j,k,th] = (phi_c_1 - phi_c_2)/(theta_1 - theta_2) 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


In [179]:
l_aa

array([[[[0.00135625, 0.00305109, 0.00349374, ..., 0.00146086,
          0.00246784, 0.00376685],
         [0.00136143, 0.00295903, 0.00365081, ..., 0.00141698,
          0.00242574, 0.00374071],
         [0.00133511, 0.00285408, 0.00284859, ..., 0.00113101,
          0.00248074, 0.00335003],
         ...,
         [0.00107096, 0.00248915, 0.00063635, ..., 0.0004812 ,
          0.00141095, 0.00132385],
         [0.00104716, 0.00247211, 0.00060933, ..., 0.00046707,
          0.00131128, 0.00139584],
         [0.00102872, 0.00244495, 0.00050108, ..., 0.00045342,
          0.00122825, 0.00114873]],

        [[0.00136569, 0.00306871, 0.00341683, ..., 0.00140406,
          0.00259963, 0.00360574],
         [0.00133409, 0.00295043, 0.003505  , ..., 0.00144108,
          0.00241166, 0.00371055],
         [0.00133363, 0.00286842, 0.00250321, ..., 0.00130662,
          0.00227753, 0.00284158],
         ...,
         [0.00112578, 0.00236418, 0.00051721, ..., 0.00048344,
          0.00131512, 0.0

In [175]:
point

(-190.0000000000001, -311.7691453623979, 0)

In [77]:

fig,axes = plt.subplots(nrows=2, sharex=True)
axes[0].plot(data['angleVec'][0],  data['fluxD'][10,10,:],label='omega')

axes[1].plot(data['angleVec'][0],  data['fluxQ'][10,10,:],label='omega')

#axes[1].plot(grid.Time,  grid.get_values('p_e_f_5')/1e6,label='omega')

#axes.plot(grid.Time,  grid.get_values('omega_5')-grid.get_values('omega_coi'),label='omega')
#axes.plot(grid.Time,  grid.get_values('P_5'),label='P_5')

axes[0].grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
data['angleVec'][0]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
      dtype=uint8)

In [50]:
data['torque'][0,0,:]

array([461.04655919, 419.51713926, 382.24325587, 348.42625386,
       336.66929706, 344.82692458, 369.37339901, 399.70419846,
       411.92991736, 395.95926812, 368.25854118, 351.93638803,
       358.24958202, 376.97017287, 399.46655516, 461.04655919,
       419.51713926, 382.24325587, 348.42625386, 336.66929706,
       344.82692458, 369.37339901, 399.70419846, 411.92991736,
       395.95926812, 368.25854118, 351.93638803, 358.24958202,
       376.97017287, 399.46655516, 461.04655919])

In [211]:
theta = data['angleVec'][0] 
phi_d = data['fluxD']
phi_q = data['fluxQ']
i_d = data['idVec'][0]
i_q = data['iqVec'][0]
i_a = I_a
i_b = I_b
i_c = I_c

np.savez('pmsm_matrices',i_d=i_d,i_q=i_q,theta=theta,
         i_a=i_a,i_b=i_b,i_c=i_c,
         phi_d=phi_d,phi_q=phi_q,
         l_aa=l_aa,l_ab=l_ab,l_ac=l_ac,l_ba=l_ba,l_bb=l_bb,l_bc=l_bc,l_ca=l_ca,l_cb=l_cb,l_cc=l_cc,
         phi_r_a_s=phi_r_a_s,phi_r_b_s=phi_r_b_s,phi_r_c_s=phi_r_c_s)

In [200]:
data['iqVec'][0][10]

0

In [205]:
data['fluxD'][0,0,4]

-0.080991824528

In [103]:
f_list = [sol[di_a],sol[di_b],sol[di_c]]
x_list = [      i_a,      i_b,     i_c]
g_list = []
y_list = []
u_dict = {}

u_dict.update({'theta_r':0.0,'omega_r':0.0})
u_dict.update({'v_a':0.0,'v_b':0.0,'v_c':0.0})

u_dict.update({'l_aa':0.1,'l_ab':0.1,'l_ac':0.1})
u_dict.update({'l_ba':0.1,'l_bb':0.1,'l_bc':0.1})
u_dict.update({'l_ca':0.1,'l_cb':0.1,'l_cc':0.1})
u_dict.update({'phi_ar':0.0,'phi_br':0.0,'phi_cr':0.0})
u_dict.update({'tau':0.0})

params_dict = {'R_s':0.01}


h_dict = {'tau':tau}

In [104]:
import numpy as np
import sympy as sym
import pydae.build_cffi as db

In [105]:
sys = {'name':'pmsm',
       'params_dict':params_dict,
       'f_list':f_list,
       'g_list':g_list,
       'x_list':x_list,
       'y_ini_list':y_list,
       'y_run_list':y_list,
       'u_run_dict':u_dict,
       'u_ini_dict':u_dict,
       'h_dict':h_dict}

bldr = db.builder(sys)
bldr.build()

system without algebraic equations, adding dummy algebraic equation


In [102]:
print(bldr.source)

void f_ini_eval(double *out,double *x,double *y,double *u,double *p,double Dt){

out[0] = (-p[0]*x[0]*u[6]*u[10] + p[0]*x[0]*u[7]*u[9] + p[0]*x[1]*u[3]*u[10] - p[0]*x[1]*u[4]*u[9] - p[0]*x[2]*u[3]*u[7] + p[0]*x[2]*u[4]*u[6] - u[3]*u[7]*u[1]*u[13] + u[3]*u[7]*v_c + u[3]*u[10]*u[1]*u[12] - u[3]*u[10]*v_b + u[4]*u[6]*u[1]*u[13] - u[4]*u[6]*v_c - u[4]*u[9]*u[1]*u[12] + u[4]*u[9]*v_b - u[6]*u[10]*u[1]*u[11] + u[6]*u[10]*v_a + u[7]*u[9]*u[1]*u[11] - u[7]*u[9]*v_a)/(u[2]*u[6]*u[10] - u[2]*u[7]*u[9] - u[3]*u[5]*u[10] + u[3]*u[7]*u[8] + u[4]*u[5]*u[9] - u[4]*u[6]*u[8]);
out[1] = (p[0]*x[0]*u[5]*u[10] - p[0]*x[0]*u[7]*u[8] - p[0]*x[1]*u[2]*u[10] + p[0]*x[1]*u[4]*u[8] + p[0]*x[2]*u[2]*u[7] - p[0]*x[2]*u[4]*u[5] + u[2]*u[7]*u[1]*u[13] - u[2]*u[7]*v_c - u[2]*u[10]*u[1]*u[12] + u[2]*u[10]*v_b - u[4]*u[5]*u[1]*u[13] + u[4]*u[5]*v_c + u[4]*u[8]*u[1]*u[12] - u[4]*u[8]*v_b + u[5]*u[10]*u[1]*u[11] - u[5]*u[10]*v_a - u[7]*u[8]*u[1]*u[11] + u[7]*u[8]*v_a)/(u[2]*u[6]*u[10] - u[2]*u[7]*u[9] - u[3]*u[5]*u[10]